<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [1]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly
!pip install numpy

In [3]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77A64KYRFGC6NEQK3KTLBYKULC -O helpers.py

--2021-12-22 04:24:31--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77A64KYRFGC6NEQK3KTLBYKULC
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1177 (1.1K) [text/plain]
Saving to: ‘helpers.py’

helpers.py          100%[===================>]   1.15K  --.-KB/s    in 0s      

2021-12-22 04:24:31 (41.3 MB/s) - ‘helpers.py’ saved [1177/1177]



Import Libs

In [72]:
import helpers
import pandas as pd
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots
import numpy as np

In [73]:
def get_tanaji_pct(df, index, n, high):
  prev_candles = helpers.get_previous_candles(df, index, n)
  min_low = prev_candles['Low'].min()
  daily_movement = high - min_low
  tanaji_pct = (daily_movement/min_low)*100
  return tanaji_pct

def get_hourly_rsi(df_15min, index):
  df_15min_temp = df_15min[:index]
  #print(index)
  #print(df_15min_temp.tail(10))
  df_60min_o = df_15min_temp['Open'].resample('60Min', offset='30Min').apply({'Open': 'first'})
  df_60min_h = df_15min_temp['High'].resample('60Min', offset='30Min').apply({'High': 'max'})
  df_60min_l = df_15min_temp['Low'].resample('60Min', offset='30Min').apply({'Low': 'min'})
  df_60min_c = df_15min_temp['Close'].resample('60Min', offset='30Min').apply({'Close': 'last'})
  df_60min_vol = df_15min_temp['Volume'].resample('60Min', offset='30Min').apply({'Volume': 'sum'})
  
  df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l, df_60min_c, df_60min_vol], axis=1)
  df_60min.dropna(subset=['Open'], inplace=True)

  rsi = RSIIndicator(df_60min['Close']).rsi()
  df_60min = df_60min.assign(rsi=rsi.values)
  return df_60min

## Load 15 min and 60 min stock data and calculate RSI

In [74]:
file_15min = '/content/stock_data/INFY21DECFUT-HIST-15M.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-20 15:00:00+05:30,1800.90,1807.4,1800.80,1805.35,302400,46.210919
2021-12-20 15:15:00+05:30,1805.50,1806.8,1801.80,1804.60,380100,45.378938
2021-12-21 09:15:00+05:30,1808.65,1826.1,1807.90,1823.30,723000,63.179210
2021-12-21 09:30:00+05:30,1823.60,1826.0,1818.85,1825.35,297900,64.543356
2021-12-21 09:45:00+05:30,1825.35,1832.2,1825.05,1826.95,320400,65.614132


## Set Strategy Parameters

In [75]:
window_start = '2021-11-01 00:00:00'
window_end = '2021-12-15 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 300
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [76]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

df.head()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-03 10:30:00+05:30,1717.00,1725.20,1717.00,1723.95,15300,72.215216
2021-11-03 10:45:00+05:30,1723.95,1725.90,1721.20,1725.90,8700,73.816862
2021-11-03 11:00:00+05:30,1725.90,1727.15,1724.70,1727.00,3300,74.702743
2021-11-03 11:30:00+05:30,1724.30,1726.00,1721.05,1726.00,3900,70.227417
2021-11-08 11:30:00+05:30,1727.00,1729.50,1725.30,1729.20,9600,70.009015



### Step 4: Volume check

In [77]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = helpers.get_previous_candles(curr_window_df, index, back_candles)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-11-03 10:30:00+05:30,1717.00,1725.20,1717.00,1723.95,15300.0,72.215216,7020.0
2021-11-08 12:30:00+05:30,1740.30,1744.20,1740.30,1743.55,30900.0,79.843545,12180.0
2021-11-08 12:45:00+05:30,1744.60,1749.75,1743.55,1748.00,36300.0,82.269631,17340.0
2021-11-12 09:15:00+05:30,1750.65,1762.30,1746.95,1759.80,56700.0,75.893828,13080.0
2021-11-12 12:15:00+05:30,1763.15,1767.85,1762.60,1765.95,30900.0,74.522828,9420.0
2021-11-12 12:30:00+05:30,1765.95,1770.80,1765.40,1770.00,39900.0,77.318708,12900.0
2021-11-16 10:30:00+05:30,1803.95,1812.00,1803.40,1810.55,88800.0,70.101198,35400.0
2021-12-03 09:15:00+05:30,1772.00,1787.90,1770.50,1779.35,1630500.0,78.972987,325140.0
2021-12-08 09:15:00+05:30,1740.05,1760.85,1740.05,1757.65,1507200.0,75.769555,205080.0


### Step 5: Hourly candle RSI check

In [78]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  df_60min = get_hourly_rsi(df_15min, index)
  #print(df_60min.tail(15))
  
  rsi = df_60min.iloc[-1]['rsi']
  if(rsi > rsi_60min):
    # This 15min candle is eligible for signal
    print('Hourly candle RSI is greater than 50..', rsi)
    temp_df.loc[index] = row

df = temp_df
df


Hourly candle RSI is greater than 50.. 57.065264788699814
Hourly candle RSI is greater than 50.. 66.61942875078235
Hourly candle RSI is greater than 50.. 68.37599811113007
Hourly candle RSI is greater than 50.. 62.63932509496794
Hourly candle RSI is greater than 50.. 63.85582455219116
Hourly candle RSI is greater than 50.. 65.4580602758297
Hourly candle RSI is greater than 50.. 71.39341868571773
Hourly candle RSI is greater than 50.. 74.87571909925822
Hourly candle RSI is greater than 50.. 63.19859302437936


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-03 10:30:00+05:30,1717.00,1725.20,1717.00,1723.95,15300.0,72.215216
2021-11-08 12:30:00+05:30,1740.30,1744.20,1740.30,1743.55,30900.0,79.843545
2021-11-08 12:45:00+05:30,1744.60,1749.75,1743.55,1748.00,36300.0,82.269631
2021-11-12 09:15:00+05:30,1750.65,1762.30,1746.95,1759.80,56700.0,75.893828
2021-11-12 12:15:00+05:30,1763.15,1767.85,1762.60,1765.95,30900.0,74.522828
2021-11-12 12:30:00+05:30,1765.95,1770.80,1765.40,1770.00,39900.0,77.318708
2021-11-16 10:30:00+05:30,1803.95,1812.00,1803.40,1810.55,88800.0,70.101198
2021-12-03 09:15:00+05:30,1772.00,1787.90,1770.50,1779.35,1630500.0,78.972987
2021-12-08 09:15:00+05:30,1740.05,1760.85,1740.05,1757.65,1507200.0,75.769555


### Step 6: Stock movement check

In [79]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  tanaji_pct = get_tanaji_pct(df_15min, index, 5, row['High'])
  print('tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

df = temp_df
temp_df

tanaji_pct: 0.7710280373831803
tanaji_pct: 1.1951728939429183
tanaji_pct: 1.417144844374894
tanaji_pct: 2.1238373946049234
tanaji_pct: 0.563155948690228
tanaji_pct: 0.7109139509753739
tanaji_pct: 1.4756531235124413
tanaji_pct: 2.6732133115111947
tanaji_pct: 2.673469387755097


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-03 10:30:00+05:30,1717.00,1725.20,1717.00,1723.95,15300.0,72.215216
2021-11-08 12:30:00+05:30,1740.30,1744.20,1740.30,1743.55,30900.0,79.843545
2021-11-08 12:45:00+05:30,1744.60,1749.75,1743.55,1748.00,36300.0,82.269631
2021-11-12 09:15:00+05:30,1750.65,1762.30,1746.95,1759.80,56700.0,75.893828
2021-11-12 12:15:00+05:30,1763.15,1767.85,1762.60,1765.95,30900.0,74.522828
2021-11-12 12:30:00+05:30,1765.95,1770.80,1765.40,1770.00,39900.0,77.318708
2021-11-16 10:30:00+05:30,1803.95,1812.00,1803.40,1810.55,88800.0,70.101198
2021-12-03 09:15:00+05:30,1772.00,1787.90,1770.50,1779.35,1630500.0,78.972987
2021-12-08 09:15:00+05:30,1740.05,1760.85,1740.05,1757.65,1507200.0,75.769555


### Step 7: Stop Loss check

In [80]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  prev_low = prev_candle.iloc[0]['Low']
  if((row['High'] - prev_low)*lot_size > stop_loss):
    print(index, 'Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
  else:
    print(index, 'Stop Loss within range', (row['High'] - prev_low)*lot_size)
    temp_df.loc[index] = row

df = temp_df
df

2021-11-03 10:30:00+05:30 Stop Loss within range 2714.9999999999864
2021-11-08 12:30:00+05:30 Stop Loss within range 1770.0000000000273
2021-11-08 12:45:00+05:30 Stop Loss within range 2835.0000000000136
2021-11-12 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6854.999999999973
2021-11-12 12:15:00+05:30 Stop Loss within range 1754.9999999999727
2021-11-12 12:30:00+05:30 Stop Loss within range 2460.0000000000136
2021-11-16 10:30:00+05:30 Stop Loss within range 3600.0
2021-12-03 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 11940.000000000055
2021-12-08 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 13139.999999999985


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-11-03 10:30:00+05:30,1717.00,1725.20,1717.00,1723.95,15300.0,72.215216
2021-11-08 12:30:00+05:30,1740.30,1744.20,1740.30,1743.55,30900.0,79.843545
2021-11-08 12:45:00+05:30,1744.60,1749.75,1743.55,1748.00,36300.0,82.269631
2021-11-12 12:15:00+05:30,1763.15,1767.85,1762.60,1765.95,30900.0,74.522828
2021-11-12 12:30:00+05:30,1765.95,1770.80,1765.40,1770.00,39900.0,77.318708
2021-11-16 10:30:00+05:30,1803.95,1812.00,1803.40,1810.55,88800.0,70.101198


## **Final Signal**

In [81]:
for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-11-03 10:30:00+05:30 , rsi: 72.21521584458274 Entry Price 1725.2 Stop Loss: 1716.15
For candle: 2021-11-08 12:30:00+05:30 , rsi: 79.84354522555607 Entry Price 1744.2 Stop Loss: 1738.3
For candle: 2021-11-08 12:45:00+05:30 , rsi: 82.26963093624107 Entry Price 1749.75 Stop Loss: 1740.3
For candle: 2021-11-12 12:15:00+05:30 , rsi: 74.52282752299797 Entry Price 1767.85 Stop Loss: 1762.0
For candle: 2021-11-12 12:30:00+05:30 , rsi: 77.31870796144221 Entry Price 1770.8 Stop Loss: 1762.6
For candle: 2021-11-16 10:30:00+05:30 , rsi: 70.10119829742845 Entry Price 1812.0 Stop Loss: 1800.0


In [82]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
sig_param_col1 = ['Time','Entry INR','StopLoss INR','Daily Mov %','Volume','Mean Vol','15Min RSI','Horuly RSI']
for index, row in df.iterrows():
  prev_candles = helpers.get_previous_candles(df_15min, index, back, True)
  next_candles = helpers.get_next_candles(df_15min, index, next)
  
  candles = pd.concat([prev_candles, next_candles])
  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  # Get all strategy params for this signal
  stop_loss_candle = helpers.get_previous_candles(df_15min, index, 1)
  vol_back_candles_df = helpers.get_previous_candles(df_15min
                                                 , index, back_candles)
  sig_stop_loss = stop_loss_candle.iloc[0]['Low']
  sig_entry = candles['High'][back]
  sig_daily_mov_pct = np.round(get_tanaji_pct(df_15min, index, 5, row['High']),2)
  sig_rsi = candles['rsi'][back]
  sig_hourly_rsi = np.round(get_hourly_rsi(df_15min, index).iloc[-1]['rsi'],2)
  sig_mean_volume = vol_back_candles_df['Volume'].mean()
  sig_volume = row['Volume']

  fig = make_subplots(rows=2, cols=2, shared_xaxes=False,
               subplot_titles=('OHLC', 'Signal Params', 'Volume & RSI',''), 
               vertical_spacing=0.1, 
               horizontal_spacing=0.01,
               row_width=[0.5, 0.7],
               column_widths=[0.8,0.2],
               specs=[[{"secondary_y": False, "type": "candlestick"},{"secondary_y": False, "type":"table"}], 
                      [{"secondary_y": True}, {"secondary_y": False}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_entry,
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_entry, y1=sig_entry, 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_stop_loss, y1=sig_stop_loss, 
                line=dict(color='Red'),
                row=1,col=1)

  # Signal Parameters Table
  fig.add_trace(go.Table(header=dict(values=['Param','Value'],
                                     line_color='white',
                                     fill_color='darkslategray',
                                     align='left'),
                         cells=dict(values=[sig_param_col1,
                                           [index.strftime('%d-%m %H:%M'), sig_entry, 
                                            sig_stop_loss, sig_daily_mov_pct, sig_volume, 
                                            sig_mean_volume, sig_rsi, sig_hourly_rsi]],
                                    line_color='white',
                                    fill_color='black',
                                    align='left')),
                row=1, col=2)

  # RSI
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  # Volume
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  fig.add_annotation(x=back,y=sig_rsi,
                     text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=rsi_15min, y1=rsi_15min, 
                line=dict(color='Green'),
                row=2,col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1450,
    height=750,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  